# Part 6: Model Validation and Selection

https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html 

## cross validation techniques

In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100)

from sklearn.model_selection import train_test_split
from sklearn.metrics import *

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")


In [5]:
df = pd.read_csv("energy_consumption.csv")
columns=['NSM', 'T8', 'RH_8', 'T2', 'RH_9', 'RH_7', 'RH_1', 'RH_2', 'RH_3']
x = pd.DataFrame(df,columns=columns)
y = df['Appliances']
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.5, random_state=0)
x_train.shape

(9867, 9)

In [22]:
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import LeaveOneOut
cross_val_score(model,x_train,y_train,cv=LeaveOneOut(len(x_train)))

array([0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 0.,
       0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0.,
       0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0.,
       1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0.,
       0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0.,
       1., 0., 0., 1., 0.

## Bias-variance tradeoff

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    
    # calculates mean absolute error of the true and predicted values
    
    y_true,y_pred = np.array(y_true), np.array(y_pred)
    count = 0
    sum = 0
    for a,p in zip(y_true, y_pred):
        if(a != 0):
            sum += (abs(a-p)/a)
            count += 1
        return (sum/count)*100
    

## regularization(L1,L2,Elastic net)

## grid search options